In [1]:
from pynq import Overlay
from pynq import MMIO
from pynq.lib.iic import AxiIIC
import cffi, time

overlay = Overlay("/home/xilinx/pynq/overlays/axi_test.bit")

In [2]:
ADDR_BASE = 0x41600000
ADDR_RANGE = 0x10000
i2c_obj = MMIO(ADDR_BASE,ADDR_RANGE)

In [3]:
i2c_obj.write(292,1)

In [4]:
iic_ip = overlay.ip_dict['axi_iic_0']
iic = AxiIIC(iic_ip)

ffi = cffi.FFI()

In [5]:
def read_(addr,reg,length=1):
    iic.send(addr,bytes([reg]),1,0)
    buf = ffi.new("unsigned char[]",length)
    iic.receive(addr, buf, length, 0)
    return [buf[i] for i in range(length)]

def write_(addr, reg, value):
    iic.send(addr, bytes([reg,value]), 2, 0)

def to_int16(low,high):
    val = (high << 8) | low
    if val & 0x8000:
        val -= 1 << 16
    return val
print(overlay.ip_dict.keys())

dict_keys(['axi_iic_0', 'processing_system7_0'])


In [6]:
# BN0055 connection check
BN0055_addr = 0x28
chip_addr = 0x00
chip_id = read_(BN0055_addr, 0x00, 1)[0]
print("CHIP ID:", hex(chip_id)) #0xA0

CHIP ID: 0xa0


In [7]:
# roboclaw encoder 
import serial.tools.list_ports

ports = serial.tools.list_ports.comports()

for port in ports:
    print(f"Device: {port.device} | Description: {port.description}")

import sys
sys.path.append("/home/xilinx/pynq/roboclaw")  # 실제 roboclaw_3.py가 있는 경로
from roboclaw_3 import Roboclaw

rc = Roboclaw("/dev/ttyACM0", 38400)  # 또는 ttyUSB0
rc.Open()

rc.ForwardM1(0x80, 0)

encoder_1_count = rc.ReadEncM1(0x80)
print(encoder_1_count)
encoder_2_count = rc.ReadEncM2(0x80)
print(encoder_2_count)

Device: /dev/ttyACM0 | Description: USB Roboclaw 2x30A
(1, -134, 128)
(1, 0, 130)


In [8]:
#NDOF mode on 0x0C

write_(BN0055_addr, 0x3D, 0X0C)
time.sleep(0.1)

In [11]:
#calibration check
cal = read_(BN0055_addr, 0x35, 1)[0]
sys = (cal >> 6) & 0x03
gyro = (cal >> 4) & 0x03
accel = (cal >> 2) & 0x03
mag = cal & 0x03

print(f"Cal : sys = {sys} Gyro = {gyro} accel = {accel} mag = {mag}")

Cal : sys = 3 Gyro = 0 accel = 1 mag = 2


In [ ]:
while True:
    data = read_(BN0055_addr,0x08,44)
    ax = to_int16(data[0],data[1]) / 100.0
    ay = to_int16(data[2],data[3]) / 100.0
    az = to_int16(data[4],data[5]) / 100.0

    mx = to_int16(data[6],data[7]) / 16.0
    my = to_int16(data[8],data[9]) / 16.0
    mz = to_int16(data[10],data[11]) / 16.0

    gx = to_int16(data[12],data[13]) / 900.0
    gy = to_int16(data[14],data[15]) / 900.0
    gz = to_int16(data[16],data[17]) / 900.0

    heading = to_int16(data[18],data[19]) / 16.0
    roll = to_int16(data[20],data[21]) / 16.0
    pitch = to_int16(data[22],data[23]) / 16.0

    qw = to_int16(data[24],data[25]) / 16384.0
    qx = to_int16(data[26],data[27]) / 16384.0
    qy = to_int16(data[28],data[29]) / 16384.0
    qz = to_int16(data[30],data[31]) / 16384.0

    lx = to_int16(data[32],data[33]) / 100.0
    ly = to_int16(data[34],data[35]) / 100.0
    lz = to_int16(data[36],data[37]) / 100.0

    gravity_vectorx = to_int16(data[38],data[39]) / 100.0
    gravity_vectory = to_int16(data[40],data[41]) / 100.0
    gravity_vectorz = to_int16(data[42],data[43]) / 100.0

    print("------------------------------------------------------------------")
    print(f"[accel] : {ax:.2f}, {ay:.2f}, {az:.2f} m/s^2 | "
            f"Mag: ({mx:.2f}, {my:.2f}, {mz:.2f}) μT | "
            f"Gyro: ({gx:.2f}, {gy:.2f}, {gz:.2f}) °/s")
    print(f"Fusion Euler: Heading={heading:.2f}°, Roll={roll:.2f}°, Pitch={pitch:.2f}°")
    print(f"[Quaternion] w={qw:.4f}, x={qx:.4f}, y={qy:.4f}, z={qz:.4f}")
    print(f"[Linear Accel] ({lx:.2f}, {ly:.2f}, {lz:.2f}) m/s² | [Gravity] ({gravity_vectorx:.2f}, {gravity_vectory:.2f}, {gravity_vectorz:.2f}) m/s²")
    print("------------------------------------------------------------------")
    time.sleep(0.02)

------------------------------------------------------------------
[accel] : 10.05, -2.02, 4.74 m/s^2 | Mag: (1.06, -32.00, -18.25) μT | Gyro: (0.25, -1.67, 0.14) °/s
Fusion Euler: Heading=197.50°, Roll=66.56°, Pitch=47.56°
[Quaternion] w=-0.1173, x=-0.5495, y=0.2715, z=-0.7814
[Linear Accel] (0.81, 0.68, 2.41) m/s² | [Gravity] (9.04, -2.89, 2.43) m/s²
------------------------------------------------------------------
------------------------------------------------------------------
[accel] : 9.52, -2.75, 2.37 m/s^2 | Mag: (1.50, -33.06, -19.06) μT | Gyro: (-0.64, -2.93, 0.47) °/s
Fusion Euler: Heading=196.25°, Roll=68.69°, Pitch=57.06°
[Quaternion] w=-0.1094, x=-0.5681, y=0.2803, z=-0.7660
[Linear Accel] (-0.64, 0.42, -0.83) m/s² | [Gravity] (9.16, -3.07, 1.63) m/s²
------------------------------------------------------------------
------------------------------------------------------------------
[accel] : 8.28, -2.87, 1.07 m/s^2 | Mag: (1.50, -33.06, -19.06) μT | Gyro: (-0.72, -2.4

------------------------------------------------------------------
[accel] : 3.79, -0.17, -8.32 m/s^2 | Mag: (-27.56, -9.75, -33.75) μT | Gyro: (-1.28, -0.25, 0.24) °/s
Fusion Euler: Heading=265.62°, Roll=26.19°, Pitch=179.25°
[Quaternion] w=-0.1498, x=-0.7141, y=0.6622, z=-0.1703
[Linear Accel] (-0.53, -0.06, 0.47) m/s² | [Gravity] (4.33, -0.11, -8.79) m/s²
------------------------------------------------------------------
------------------------------------------------------------------
[accel] : 4.47, -0.37, -8.11 m/s^2 | Mag: (-27.75, -8.25, -34.69) μT | Gyro: (-1.48, -0.34, -0.52) °/s
Fusion Euler: Heading=265.81°, Roll=25.56°, Pitch=-178.25°
[Quaternion] w=-0.1608, x=-0.7142, y=0.6639, z=-0.1528
[Linear Accel] (0.24, -0.63, 0.72) m/s² | [Gravity] (4.23, 0.26, -8.84) m/s²
------------------------------------------------------------------
------------------------------------------------------------------
[accel] : 4.46, 0.41, -9.19 m/s^2 | Mag: (-27.75, -8.25, -34.69) μT | Gyro: (

------------------------------------------------------------------
[accel] : 5.15, -3.13, 6.46 m/s^2 | Mag: (5.50, -27.19, -19.00) μT | Gyro: (3.21, 3.51, 0.90) °/s
Fusion Euler: Heading=196.19°, Roll=41.62°, Pitch=32.56°
[Quaternion] w=-0.1274, x=-0.3328, y=0.2727, z=-0.8937
[Linear Accel] (-1.17, 0.60, -0.20) m/s² | [Gravity] (6.51, -3.94, 6.17) m/s²
------------------------------------------------------------------
------------------------------------------------------------------
[accel] : 5.88, -3.11, 6.50 m/s^2 | Mag: (5.50, -27.19, -19.00) μT | Gyro: (3.29, 3.47, 0.74) °/s
Fusion Euler: Heading=194.88°, Roll=37.94°, Pitch=28.06°
[Quaternion] w=-0.1194, x=-0.3049, y=0.2435, z=-0.9130
[Linear Accel] (-0.14, 0.52, -0.31) m/s² | [Gravity] (5.78, -3.47, 7.11) m/s²
------------------------------------------------------------------
------------------------------------------------------------------
[accel] : 4.31, -2.46, 6.74 m/s^2 | Mag: (7.06, -30.19, -17.19) μT | Gyro: (3.01, 2.93, 0

------------------------------------------------------------------
[accel] : 5.84, 1.52, 6.89 m/s^2 | Mag: (3.50, -39.75, 12.00) μT | Gyro: (-0.05, 2.04, -0.13) °/s
Fusion Euler: Heading=176.56°, Roll=33.31°, Pitch=-11.25°
[Quaternion] w=0.0283, x=-0.2856, y=-0.0943, z=-0.9532
[Linear Accel] (2.06, -1.04, -0.74) m/s² | [Gravity] (5.14, 1.62, 8.18) m/s²
------------------------------------------------------------------
------------------------------------------------------------------
[accel] : 1.32, 1.97, 11.63 m/s^2 | Mag: (3.50, -39.75, 12.00) μT | Gyro: (-0.49, 2.96, 0.81) °/s
Fusion Euler: Heading=175.31°, Roll=28.12°, Pitch=-10.31°
[Quaternion] w=0.0377, x=-0.2245, y=-0.0833, z=-0.9702
[Linear Accel] (-3.06, 0.64, 1.35) m/s² | [Gravity] (4.33, 1.41, 8.68) m/s²
------------------------------------------------------------------
------------------------------------------------------------------
[accel] : 6.69, 1.13, 6.92 m/s^2 | Mag: (3.50, -39.75, 12.38) μT | Gyro: (-1.75, 2.43, -0.

------------------------------------------------------------------
[accel] : -6.00, 0.54, 7.17 m/s^2 | Mag: (2.25, -40.19, 12.69) μT | Gyro: (-0.37, 2.80, -1.34) °/s
Fusion Euler: Heading=185.81°, Roll=-41.69°, Pitch=-5.44°
[Quaternion] w=-0.0475, x=0.3542, y=-0.0564, z=-0.9323
[Linear Accel] (0.31, -0.12, -0.50) m/s² | [Gravity] (-6.52, 0.70, 7.28) m/s²
------------------------------------------------------------------
------------------------------------------------------------------
[accel] : -6.46, 0.68, 5.84 m/s^2 | Mag: (2.88, -40.25, 13.38) μT | Gyro: (-0.15, 2.61, -0.93) °/s
Fusion Euler: Heading=186.69°, Roll=-44.62°, Pitch=-4.50°
[Quaternion] w=-0.0543, x=0.3778, y=-0.0526, z=-0.9228
[Linear Accel] (0.00, 0.29, -0.64) m/s² | [Gravity] (-7.08, 0.50, 6.76) m/s²
------------------------------------------------------------------
------------------------------------------------------------------
[accel] : -8.08, 0.36, 6.96 m/s^2 | Mag: (4.75, -41.50, 12.50) μT | Gyro: (0.41, 2.89,

...

------------------------------------------------------------------
[accel] : 1.87, 1.35, 9.40 m/s^2 | Mag: (14.00, -37.38, 7.69) μT | Gyro: (-0.04, 0.08, 0.06) °/s
Fusion Euler: Heading=194.81°, Roll=12.06°, Pitch=-8.38°
[Quaternion] w=-0.1280, x=-0.1140, y=-0.0579, z=-0.9835
[Linear Accel] (-0.17, -0.04, -0.08) m/s² | [Gravity] (2.05, 1.40, 9.48) m/s²
------------------------------------------------------------------
------------------------------------------------------------------
[accel] : 1.67, 1.25, 9.55 m/s^2 | Mag: (14.88, -37.69, 5.56) μT | Gyro: (-0.04, 0.04, 0.07) °/s
Fusion Euler: Heading=194.69°, Roll=11.94°, Pitch=-8.25°
[Quaternion] w=-0.1273, x=-0.1129, y=-0.0573, z=-0.9838
[Linear Accel] (-0.29, -0.09, -0.04) m/s² | [Gravity] (2.03, 1.38, 9.49) m/s²
------------------------------------------------------------------
------------------------------------------------------------------
[accel] : 2.03, 1.41, 9.21 m/s^2 | Mag: (14.88, -37.69, 5.56) μT | Gyro: (-0.06, 0.02, -0

------------------------------------------------------------------
[accel] : 1.94, 1.14, 9.43 m/s^2 | Mag: (13.69, -38.06, 4.75) μT | Gyro: (-0.02, 0.01, -0.01) °/s
Fusion Euler: Heading=194.06°, Roll=12.50°, Pitch=-6.94°
[Quaternion] w=-0.1217, x=-0.1160, y=-0.0459, z=-0.9847
[Linear Accel] (-0.18, -0.01, -0.06) m/s² | [Gravity] (2.13, 1.16, 9.50) m/s²
------------------------------------------------------------------
------------------------------------------------------------------
[accel] : 1.96, 1.13, 9.30 m/s^2 | Mag: (13.69, -38.06, 4.75) μT | Gyro: (-0.03, -0.10, 0.02) °/s
Fusion Euler: Heading=194.06°, Roll=12.62°, Pitch=-6.94°
[Quaternion] w=-0.1216, x=-0.1169, y=-0.0455, z=-0.9846
[Linear Accel] (-0.17, -0.04, -0.11) m/s² | [Gravity] (2.14, 1.15, 9.49) m/s²
------------------------------------------------------------------
------------------------------------------------------------------
[accel] : 2.15, 1.16, 9.59 m/s^2 | Mag: (14.06, -37.69, 4.38) μT | Gyro: (0.07, 0.06, -

------------------------------------------------------------------
[accel] : 2.17, 1.02, 9.37 m/s^2 | Mag: (13.00, -36.56, 4.38) μT | Gyro: (0.06, 0.06, -0.03) °/s
Fusion Euler: Heading=194.19°, Roll=13.44°, Pitch=-6.50°
[Quaternion] w=-0.1225, x=-0.1234, y=-0.0407, z=-0.9839
[Linear Accel] (-0.06, -0.06, 0.03) m/s² | [Gravity] (2.28, 1.08, 9.47) m/s²
------------------------------------------------------------------
------------------------------------------------------------------
[accel] : 2.28, 1.05, 9.48 m/s^2 | Mag: (13.00, -36.56, 4.38) μT | Gyro: (0.03, 0.07, -0.05) °/s
Fusion Euler: Heading=194.19°, Roll=13.31°, Pitch=-6.56°
[Quaternion] w=-0.1228, x=-0.1226, y=-0.0413, z=-0.9839
[Linear Accel] (-0.01, -0.02, -0.09) m/s² | [Gravity] (2.26, 1.09, 9.47) m/s²
------------------------------------------------------------------
------------------------------------------------------------------
[accel] : 1.97, 1.03, 9.31 m/s^2 | Mag: (14.50, -37.69, 4.38) μT | Gyro: (-0.05, -0.12, 0.

------------------------------------------------------------------
[accel] : 2.35, 1.01, 9.31 m/s^2 | Mag: (14.50, -38.19, 3.06) μT | Gyro: (-0.07, -0.01, -0.02) °/s
Fusion Euler: Heading=194.44°, Roll=13.75°, Pitch=-6.38°
[Quaternion] w=-0.1246, x=-0.1263, y=-0.0390, z=-0.9833
[Linear Accel] (0.01, -0.05, -0.15) m/s² | [Gravity] (2.34, 1.06, 9.46) m/s²
------------------------------------------------------------------
------------------------------------------------------------------
[accel] : 2.15, 0.96, 9.23 m/s^2 | Mag: (14.50, -38.19, 1.88) μT | Gyro: (-0.01, -0.07, 0.05) °/s
Fusion Euler: Heading=194.38°, Roll=13.94°, Pitch=-6.25°
[Quaternion] w=-0.1241, x=-0.1276, y=-0.0379, z=-0.9833
[Linear Accel] (-0.21, -0.07, -0.21) m/s² | [Gravity] (2.36, 1.04, 9.45) m/s²
------------------------------------------------------------------
------------------------------------------------------------------
[accel] : 2.26, 1.04, 9.50 m/s^2 | Mag: (13.38, -37.38, 4.00) μT | Gyro: (0.06, 0.12, -

------------------------------------------------------------------
[accel] : 2.18, 1.00, 9.32 m/s^2 | Mag: (15.19, -37.75, 3.56) μT | Gyro: (0.03, -0.01, 0.04) °/s
Fusion Euler: Heading=194.56°, Roll=14.00°, Pitch=-6.19°
[Quaternion] w=-0.1260, x=-0.1278, y=-0.0370, z=-0.9831
[Linear Accel] (-0.18, -0.02, -0.13) m/s² | [Gravity] (2.37, 1.03, 9.45) m/s²
------------------------------------------------------------------
------------------------------------------------------------------
[accel] : 2.29, 1.05, 9.60 m/s^2 | Mag: (13.69, -37.75, 4.00) μT | Gyro: (0.12, 0.17, -0.06) °/s
Fusion Euler: Heading=194.62°, Roll=13.75°, Pitch=-6.38°
[Quaternion] w=-0.1264, x=-0.1260, y=-0.0388, z=-0.9832
[Linear Accel] (0.00, 0.03, 0.14) m/s² | [Gravity] (2.33, 1.06, 9.46) m/s²
------------------------------------------------------------------
------------------------------------------------------------------
[accel] : 2.10, 1.08, 9.46 m/s^2 | Mag: (13.69, -37.75, 4.00) μT | Gyro: (-0.08, -0.08, 0.07